In [ ]:
 #This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-may-2022/train.csv")
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-may-2022/test.csv")

In [ ]:
#set print options to see all columns
pd.set_option('display.max_columns', None)
train_df.head()

In [ ]:
tr_df1 = train_df.drop(columns=["id", "f_27"], axis=1)
ts_df1 = test_df.drop(columns=["id", "f_27"], axis=1)

In [ ]:
tr_df1.shape

In [ ]:
tr_df1.describe()

# Base Model

In [ ]:
df1_X = tr_df1.iloc[:, :30]
df1_X.shape

In [ ]:
df1_y = tr_df1.iloc[:, 30]
df1_y.shape

In [ ]:
df1_y.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df1_X_scaled = scaler.fit_transform(df1_X)

In [ ]:
df1_y = df1_y.to_numpy()

In [ ]:
#splitting the train_df further
from sklearn.model_selection import train_test_split
(df1_X_train, df1_X_test, df1_y_train, df1_y_test) = train_test_split(df1_X_scaled, df1_y)
df1_X_train.shape

In [ ]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(df1_X_train, df1_y_train)

In [ ]:
y_preds = xgb_clf.predict(df1_X_test)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
auc = roc_auc_score(y_preds, df1_y_test)
auc

In [ ]:
import matplotlib.pyplot as plt

#plotting auc roc curve
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(df1_y_test, y_preds)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc, estimator_name="base_xgb")

display.plot()
plt.show()

In [ ]:
#base neural network
#splitting dataset into training and validation sets
X_t, X_v, y_t, y_v = train_test_split(df1_X_train, df1_y_train)

In [ ]:
import tensorflow as tf
from tensorflow import keras

#creating a Sequential neural network
model = keras.models.Sequential([
                              keras.layers.Dense(64, activation='relu', input_shape=X_t.shape[1:]),
                              keras.layers.Dense(128, activation='relu'),
                              keras.layers.Dense(128, activation='relu'),
                              keras.layers.Dense(1, activation='sigmoid')
                                
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ["accuracy"]
)

In [ ]:
history = model.fit(
    X_t, y_t, epochs=10,
    validation_data = (X_v, y_v)
)

In [ ]:
y_preds = model.predict(df1_X_test)

In [ ]:
#calculating auc score for nn, because predict method produces probabilities
auc = roc_auc_score(y_preds.round().astype(int), df1_y_test)
auc

In [ ]:
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) #set the vertical limits [0, 1]
plt.show()

In [ ]:
y_preds[:10].round().astype(int)

In [ ]:
df1_y_test[:10]

In [ ]:
#scaling the test set,, ts_df1
ts_df1_scaled = scaler.fit_transform(ts_df1)

In [ ]:
ts_df1_scaled.shape

In [ ]:
test_preds = model.predict(ts_df1_scaled)
test_preds[:10]

In [ ]:
# Save predictions in format used for competition scoring
output = pd.DataFrame({'id': test_df.id,
                       'target': test_preds.reshape(1,-1).flatten().tolist()})
output.to_csv('submission.csv', index=False)